In [1]:
import numpy as np
import scipy.io

from matplotlib import pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential, Input, Model
from keras.layers.core import Dense, Dropout, Activation, Reshape, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
mat = scipy.io.loadmat('data/train_32X32.mat')
mat = mat['X']
b, h, d, n = mat.shape

#### GreyScale

In [3]:
#Convert all RGB-Images to greyscale
img_gray = np.zeros(shape =(n, b, h, 1))

def rgb2gray(rgb): 
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

for i in range(n):
    #Convert to greyscale
    img = rgb2gray(mat[:,:,:,i])
    img = img.reshape(1, 32, 32, 1)
    img_gray[i,:] = img

# Normalize input
img_gray = img_gray/255.

#### Model creation

In [4]:
img_size = Input(shape=(b, h, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(img_size)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(img_size, decoded)
autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')#, metrics=['binary_accuracy'])

# Output summary of network
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           584       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 8)           0         
__________

In [5]:

callbacks = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
n_epochs = 10
batch_size = 128

autoencoder.fit( img_gray, img_gray, epochs=n_epochs, batch_size=batch_size,shuffle=True, validation_split=0.2)

Train on 58605 samples, validate on 14652 samples
Epoch 1/10
35584/58605 [=================>............] - ETA: 2:47 - loss: 0.6419